<a href="https://colab.research.google.com/github/piotrlnordea/AmazonQA/blob/main/embedd1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install sentence-transformers transformers



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:


import pandas as pd

# Step 1: Read the CSV file
data = pd.read_csv('bitex.csv')

# Step 2: Display the first few rows of the data
print("First few rows of the data:")
print(data.head())

# Step 3: Get information about the data
print("\nData information:")
print(data.info())

# Step 4: Summary statistics
print("\nSummary statistics:")
print(data.describe())

# Step 5: Unique values in categorical columns
categorical_cols = ['flags', 'utterance', 'category', 'intent']
for col in categorical_cols:
    unique_values = data[col].unique()
    print(f"\nUnique values in '{col}':")
    print(unique_values)
    if col=='category':
      uv_cat=unique_values
    if col=='intent':
      uv_int=unique_values

# Step 6: Value counts of categorical columns
for col in categorical_cols:
    value_counts = data[col].value_counts()
    print(f"\nValue counts for '{col}':")
    print(value_counts)

# Step 7: Check for missing values
missing_values = data.isnull().sum()
print("\nMissing values:")
print(missing_values)
print(uv_cat)
print(uv_int)

First few rows of the data:
   flags                                          utterance category  \
0   BILC  I don't have an online account, what do I have...  ACCOUNT   
1  BILQZ  can you tell me if i can regisger two accounts...  ACCOUNT   
2   BPLC         I have no online account, open one, please  ACCOUNT   
3  BIPLD  could you ask an agent how to open an account,...  ACCOUNT   
4   BLQC               i want an online account, create one  ACCOUNT   

           intent  
0  create_account  
1  create_account  
2  create_account  
3  create_account  
4  create_account  

Data information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21534 entries, 0 to 21533
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   flags      21534 non-null  object
 1   utterance  21534 non-null  object
 2   category   21534 non-null  object
 3   intent     21534 non-null  object
dtypes: object(4)
memory usage: 673.1+ KB
None

Summary st

In [39]:
import tensorflow as tf
import tensorflow_hub as hub
from sentence_transformers import SentenceTransformer

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model_use = hub.load(module_url)

model_st1 = SentenceTransformer('all-mpnet-base-v2')
model_st2 = SentenceTransformer('all-MiniLM-L6-v2')
model_st3 = SentenceTransformer('all-distilroberta-v1')

In [40]:
def embed(model, model_type, sentences):
    if model_type == 'use':
        embeddings = model(sentences)
    elif model_type == 'sentence transformer':
        embeddings = model.encode(sentences)

    return embeddings

In [41]:
print(uv_cat)

['ACCOUNT' 'CANCELLATION_FEE' 'CONTACT' 'DELIVERY' 'FEEDBACK' 'INVOICES'
 'NEWSLETTER' 'ORDER' 'PAYMENT' 'REFUNDS' 'SHIPPING']


In [42]:
all_intents = list(data['utterance'])
len(all_intents)
embeddings_use = embed(model_use, 'use', all_intents)
embeddings_use.shape

TensorShape([21534, 512])

In [43]:


embeddings_use0c = embed(model_use, 'use', uv_cat)

embeddings_use0c.shape

TensorShape([11, 512])

In [44]:
embeddings_use0i = embed(model_use, 'use', uv_int)
embeddings_use0i.shape

TensorShape([27, 512])

In [54]:
import numpy as np
# Compute cosine similarity between each row in A and B


cos_sim = np.dot(embeddings_use.numpy(), embeddings_use0c.numpy().T)

# Find the closest row in B for each row in A
closest_indices = np.argmax(cos_sim, axis=1)
#closest_rows = embeddings_use0c[closest_indices]
print(closest_indices)
#print("Closest rows in B for each row in A:")
print("Closest rows in B for each row in A:")

#print(closest_rows)



[ 0  0  0 ... 10 10 10]
Closest rows in B for each row in A:


In [46]:
closest_rows = uv_cat[closest_indices]
print(closest_rows)

['ACCOUNT' 'ACCOUNT' 'ACCOUNT' ... 'SHIPPING' 'SHIPPING' 'SHIPPING']


In [47]:
data['embcat'] = closest_rows
print(data)

       flags                                          utterance  category  \
0       BILC  I don't have an online account, what do I have...   ACCOUNT   
1      BILQZ  can you tell me if i can regisger two accounts...   ACCOUNT   
2       BPLC         I have no online account, open one, please   ACCOUNT   
3      BIPLD  could you ask an agent how to open an account,...   ACCOUNT   
4       BLQC               i want an online account, create one   ACCOUNT   
...      ...                                                ...       ...   
21529  BILQC  i have no shipping address, what do i have to ...  SHIPPING   
21530   BLQC  I have no shipping address and I want to set o...  SHIPPING   
21531   BIQC  i want to set up my shipping address, what do ...  SHIPPING   
21532   BILC  I don't have a shipping address, can you set o...  SHIPPING   
21533    BQZ                i  wanna set up my shipping address  SHIPPING   

                        intent    embcat  
0               create_account  

In [52]:

num_diff_rows = data[data['category'] != data['embcat']]

# Print the result
print("Number of rows where values in 'Column1' and 'Column2' differ:", num_diff_rows.shape[0])


Number of rows where values in 'Column1' and 'Column2' differ: 4494


In [53]:
print(num_diff_rows.head(10))

      flags                                          utterance category  \
47    BILCZ           I want a profille, how can I create one?  ACCOUNT   
135    BLCZ  I need an online accoung and I want to creaet one  ACCOUNT   
272      BL                                    how to register  ACCOUNT   
305   BILCZ  I want to create a online accountg, can you he...  ACCOUNT   
321  BILQCZ      I want an online zccount, how can I open one?  ACCOUNT   
570   BILCZ          I have no prfofile, what do I have to do?  ACCOUNT   
577    BPLZ                    help me create a profile,please  ACCOUNT   
660   BILCZ            I want a user avcount, can you help me?  ACCOUNT   
788   BILCZ                  I need a profile,can you help me?  ACCOUNT   
800   BILQZ                           can I create  a provile?  ACCOUNT   

             intent            embcat  
47   create_account             ORDER  
135  create_account  CANCELLATION_FEE  
272  create_account          FEEDBACK  
305  create_ac

In [49]:
embeddings_st1 = embed(model_st1, 'sentence transformer', all_intents)
embeddings_st1.shape

(21534, 768)

In [50]:
embeddings_st2 = embed(model_st2, 'sentence transformer', all_intents)
embeddings_st2.shape

(21534, 384)

In [ ]:
embeddings_st3 = embed(model_st3, 'sentence transformer', all_intents)
embeddings_st3.shape






(21534, 768)